In [2]:
# 本文所用评价指标参考博客：
# https://blog.csdn.net/sinat_26917383/article/details/70577710

In [1]:
import scipy.io
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
# ORL人脸数据库由该实验室从1992年4月到1994年4月期间拍摄的一系列人脸图像组成，
# 共有40个不同年龄、不同性别和不同种族的对象。
# 每个人10幅图像共计400幅灰度图像组成，图像尺寸是92×112，图像背景为黑色

In [3]:
os.getcwd()

'C:\\Users\\Tianh\\Desktop\\fth'

In [4]:
ls

 驱动器 C 中的卷是 OS
 卷的序列号是 7A99-DFFA

 C:\Users\Tianh\Desktop\fth 的目录

2019/10/25  17:39    <DIR>          .
2019/10/25  17:39    <DIR>          ..
2019/10/20  18:36    <DIR>          .ipynb_checkpoints
2019/10/25  17:39            23,106 data_experiment_ORL_Data.ipynb
2019/10/20  19:03            15,460 data_experiment_WINE_white_Data.ipynb
2019/10/19  09:47            54,806 data_experiment_WINR_red_Data.ipynb
2019/10/20  19:07            18,030 data_experiment_Yale_Data.ipynb
2019/10/18  17:50            13,071 Untitled.ipynb
2019/10/18  18:23            49,825 Untitled1.ipynb
2019/10/19  09:49               586 Untitled2.ipynb
2019/10/20  18:48             2,833 Untitled3.ipynb
               8 个文件        177,717 字节
               3 个目录 245,594,181,632 可用字节


In [5]:
#导入数据

In [6]:
# data = scipy.io.loadmat('C:\\Users\\Lab\\Desktop\\DMLab\\data\\face\\ORL_32x32.mat')
data = scipy.io.loadmat('C:\\Users\\Tianh\\Desktop\\DMLab\\data\\face\\ORL_32x32.mat')

In [7]:
type(data)
data
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'gnd', 'fea'])

In [8]:
#对数据进行预处理

In [9]:
fea = data['fea']

In [10]:
fea

array([[ 75,  83,  81, ...,  35,  40,  39],
       [110, 120, 130, ...,  34,  40,  33],
       [ 77,  65,  49, ...,  38,  36,  35],
       ...,
       [121, 113, 118, ...,  42,  42,  38],
       [ 52,  51,  61, ..., 111, 132, 141],
       [125, 125, 120, ...,  82,  88,  87]], dtype=uint8)

In [11]:
print(fea.shape)
print(type(fea))
print(fea.dtype)
fea = data['fea'].astype(np.float64)
print(fea.dtype)
print(fea)

(400, 1024)
<class 'numpy.ndarray'>
uint8
float64
[[ 75.  83.  81. ...  35.  40.  39.]
 [110. 120. 130. ...  34.  40.  33.]
 [ 77.  65.  49. ...  38.  36.  35.]
 ...
 [121. 113. 118. ...  42.  42.  38.]
 [ 52.  51.  61. ... 111. 132. 141.]
 [125. 125. 120. ...  82.  88.  87.]]


In [12]:
# print(data['gnd'])
print(type(data['gnd']))
print(data['gnd'].shape)
true_label= data['gnd'].ravel()  #降维 返回视图
# print(gnd)
print(type(true_label))
print(true_label.shape)
print(true_label)

<class 'numpy.ndarray'>
(400, 1)
<class 'numpy.ndarray'>
(400,)
[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10 11 11 11 11 11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12
 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 14 14 15 15 15 15
 15 15 15 15 15 15 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17
 17 17 18 18 18 18 18 18 18 18 18 18 19 19 19 19 19 19 19 19 19 19 20 20
 20 20 20 20 20 20 20 20 21 21 21 21 21 21 21 21 21 21 22 22 22 22 22 22
 22 22 22 22 23 23 23 23 23 23 23 23 23 23 24 24 24 24 24 24 24 24 24 24
 25 25 25 25 25 25 25 25 25 25 26 26 26 26 26 26 26 26 26 26 27 27 27 27
 27 27 27 27 27 27 28 28 28 28 28 28 28 28 28 28 29 29 29 29 29 29 29 29
 29 29 30 30 30 30 30 30 30 30 30 30 31 31 31 31 31 31 31 31

In [13]:
#对数据进行归一化

In [14]:
#最大最小值归一化
# 参考博客：https://blog.csdn.net/zlzl8885/article/details/78094366
def minmax_Norm(dataSet):
    minVals = dataSet.min(0) # 取每一列的最小值
    maxVals = dataSet.max(0) # 取每一列的最大值
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals, (m, 1))
    normDataSet = normDataSet/np.tile(ranges, (m, 1))   
    return normDataSet

In [15]:
# Z-score归一化
# 参考博客：https://blog.csdn.net/sinat_36458870/article/details/79498302#2Zscore_22
def zscore_Norm(dataSet):  
    mu = dataSet.average()
    sigma = dataSet.std()
    x = (x - mu) / sigma
    return datsSet  

In [16]:
# Sigmoid归一化
# Sigmoid函数是一个具有S形曲线的函数，
# 是良好的阈值函数，在(0, 0.5)处中心对称，
# 在(0, 0.5)附近有比较大的斜率而当数据趋向于正无穷和负无穷的时候，
# 映射出来的值就会无限趋向于1和0
def sigmoid_Norm(dataSet,useStatus):  
    if useStatus:  
        return 1.0 / (1 + np.exp(-float(dataSet)))
    else:  
        return float(dataSet)
# useStatus管理是否使用sigmoid的状态，方便调试使用

In [17]:
norm_fea = minmax_Norm(fea)
norm_fea

array([[0.31034483, 0.33879781, 0.31868132, ..., 0.13366337, 0.140625  ,
        0.12121212],
       [0.51149425, 0.54098361, 0.58791209, ..., 0.12871287, 0.140625  ,
        0.09090909],
       [0.32183908, 0.24043716, 0.14285714, ..., 0.14851485, 0.11979167,
        0.1010101 ],
       ...,
       [0.57471264, 0.50273224, 0.52197802, ..., 0.16831683, 0.15104167,
        0.11616162],
       [0.17816092, 0.16393443, 0.20879121, ..., 0.50990099, 0.61979167,
        0.63636364],
       [0.59770115, 0.56830601, 0.53296703, ..., 0.36633663, 0.390625  ,
        0.36363636]])

In [18]:
#k-means 聚类

In [19]:
cluster_num = 40

In [26]:
kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(norm_fea)

In [27]:
predict_label  =kmeans.labels_
print(predict_label)

[ 9 25  9  4 19 22  9 22  9 19 15 15 15 15 15 15 15 15 15 15 32 32 32 24
 24  5  2 24 14 14 17 17 24 14 14 22 14 17 17 14  3  3  3  3  2  2  2  2
  2  7 14 14 14 16 16 16 16 16 16 16  1  1 38  2  1 38  1  1 38 21 37 37
 37 25 37 37 25 37 22 25  0  3  0  2  2  2  2  2  3  3 13 13 32 32 19 19
 19 19 22 37 27 27 27 27 27 27 27 27 27 27 19  0  0  0  0 19  0 19 19 19
 17 17  2  2 17 17 24 24 14 17 35 28 28 35 35 28 35 35 28 28 17 17 17 17
 17 17 17 17 17 17 11  9  9 14 14 11  9 22  9  9 39 39 32 32 39  4  4 39
  4  4 38 38 38 38 38 38 38 38 38 38 36 36 36  1  1 36  1  1  1 36 23 23
 23 17 22 17 22 22 22 23 23 23 23 24  3  3 24 24 17 24 30 30 30 30 30  8
 30 30 30 30 14 14 32  2 32  2  2 14  2  2 11 11 11 11 23 11 11 11 11 11
 13 32 13 13 13 13 13  2 13 13  5  5 32 32 32  5 14 32  5 32 18 18 18 18
 18 18 18 18 18 18 10 20 10 20 20 10 20 10 20 10 34 34 34 34 24 24 24 24
 34 34  3 23  3 23 23  3 23 23  3  3 12 31 31 31 31 12 12 12 12 31  1 36
 36  1 36  1 17  1  1  1 33 33 33 33 33 33 33 33 33

In [22]:
kmeans.cluster_centers_

array([[0.57799672, 0.56596409, 0.53532182, ..., 0.14002829, 0.12053571,
        0.12265512],
       [0.42672414, 0.50478142, 0.55700549, ..., 0.24659653, 0.23014323,
        0.1875    ],
       [0.35710554, 0.3219076 , 0.31843157, ..., 0.43271827, 0.41808712,
        0.38957759],
       ...,
       [0.27422003, 0.33489461, 0.40423862, ..., 0.18528996, 0.19568452,
        0.19408369],
       [0.44076039, 0.39554435, 0.38123415, ..., 0.32368621, 0.3036859 ,
        0.27894328],
       [0.42241379, 0.39617486, 0.45054945, ..., 0.19059406, 0.17317708,
        0.15025253]])

In [1]:
#K-Mediod 聚类

In [15]:
def pearson_distance(vector1, vector2):
    from scipy.spatial.distance import pdist
    X = vstack([vector1, vector2])
    d2 = pdist(X)
    return d2

In [16]:
distances_cache = {}

In [17]:
def totalcost(blogwords, costf, medoids_idx):
    size = len(blogwords)
    total_cost = 0.0
    medoids = {}
    for idx in medoids_idx:
        medoids[idx] = []
    for i in range(size):
        choice = None
        min_cost = inf
        for m in medoids:
            tmp = distances_cache.get((m, i), None)
            if tmp == None:
                tmp = pearson_distance(blogwords[m], blogwords[i])
                distances_cache[(m, i)] = tmp
            if tmp < min_cost:
                choice = m
                min_cost = tmp
        medoids[choice].append(i)
        total_cost += min_cost
    return total_cost, medoids

In [18]:
def kmedoids(blogwords, k):
    import random
    size = len(blogwords)
    medoids_idx = random.sample([i for i in range(size)], k)
    pre_cost, medoids = totalcost(blogwords, pearson_distance, medoids_idx)
#     print pre_cost
    current_cost = inf  # maxmum of pearson_distances is 2.
    best_choice = []
    best_res = {}
    iter_count = 0
    while 1:
        for m in medoids:
            for item in medoids[m]:
                if item != m:
                    idx = medoids_idx.index(m)
                    swap_temp = medoids_idx[idx]
                    medoids_idx[idx] = item
                    tmp, medoids_ = totalcost(blogwords, pearson_distance, medoids_idx)
                    # print tmp,'-------->',medoids_.keys()
                    if tmp < current_cost:
                        best_choice = list(medoids_idx)
                        best_res = dict(medoids_)
                        current_cost = tmp
                    medoids_idx[idx] = swap_temp
        iter_count += 1
#         print current_cost, iter_count
        if best_choice == medoids_idx: break
        if current_cost <= pre_cost:
            pre_cost = current_cost
            medoids = best_res
            medoids_idx = best_choice

    return current_cost, best_choice, best_res

In [19]:
best_cost, best_choice, best_medoids = kmedoids(data, 15)

ValueError: Sample larger than population or is negative

In [ ]:
data = mat(data)
listone = []

In [22]:
#聚类评价

In [23]:
#Mutual Information based scores 互信息
print(metrics.adjusted_mutual_info_score(true_label,predict_label))

# Homogeneity 同质性 每个群集只包含单个类的成员
print(metrics.homogeneity_score(true_label, predict_label))

# completeness 完整性 类的所有成员都分配给同一个群集
print(metrics.completeness_score(true_label, predict_label))

# 两者的调和平均V-measure：
print(metrics.v_measure_score(true_label, predict_label))

#  Fowlkes-Mallows scores
# The Fowlkes-Mallows score FMI is defined as the geometric mean 
# of the pairwise precision and recall:
print(metrics.fowlkes_mallows_score(true_label,predict_label))

# Silhouette Coefficient 轮廓系数
print(metrics.silhouette_score(fea, predict_label, metric='euclidean'))

# Calinski-Harabaz Index 分数值ss越大则聚类效果越好
# 类别内部数据的协方差越小越好，
# 类别之间的协方差越大越好，这样的Calinski-Harabasz分数会高。
print(metrics.calinski_harabaz_score(fea, predict_label))

0.6128035473054246
0.7626220065791752
0.7914066386234303
0.7767477396639544
0.4433977082926789
0.14294047669236798
14.849598926320414


D:\Users\Tianh\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
D:\Users\Tianh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function calinski_harabaz_score is deprecated; Function 'calinski_harabaz_score' has been renamed to 'calinski_harabasz_score' and will be removed in version 0.23.
  warnings.warn(msg, category=DeprecationWarning)


In [30]:
#一些评价指标的实现代码

In [31]:
# 交叉熵及纯度实现参考博客：
# https://blog.csdn.net/vernice/article/details/46467449

In [ ]:
#老师上课所讲

In [8]:
# scipy.io.savemat()